In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from scipy.stats import spearmanr
import pandas as pd
import requests
from bs4 import BeautifulSoup
import dcor
import plotly.express as px
import matplotlib.pyplot as plt
import cvxpy as cp

import math

import numpy as np
import pandas as pd
import scipy.stats as scs
import scipy.io 

from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import os
os.add_dll_directory('C:\Aorda\PSG\lib')
import psgpython as psg

In [2]:
tables=pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
sp500_wiki=tables[0]
sp500_wiki=sp500_wiki.iloc[:,:3]
print(sp500_wiki.shape)
sp500_wiki.head()

(503, 3)


,Symbol,Security,GICS Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie,Health Care
4,ACN,Accenture,Information Technology


In [3]:
tickers=sp500_wiki['Symbol'].unique().tolist()
tickers[:10]

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']

In [4]:
tickers = [t.replace('BF.B', 'BF-B').replace('BRK.B', 'BRK-B') for t in tickers]

In [5]:
start_date="2018-01-01"
end_date="2024-12-31"
df = yf.download(tickers  ,start=start_date, end=end_date,auto_adjust=False)
df= df['Close'].reset_index()
print(df.shape)
df.head()

[*********************100%***********************]  503 of 503 completed

503 Failed downloads:
['DRI', 'ANET', 'SNA', 'URI', 'ETR', 'NCLH', 'ADI', 'PFG', 'HLT', 'PLTR', 'MOS', 'CNP', 'AMCR', 'NEM', 'MCD', 'INVH', 'DG', 'JBL', 'SYF', 'IFF', 'FE', 'BBY', 'NRG', 'LOW', 'POOL', 'OTIS', 'CVS', 'AMAT', 'IP', 'LEN', 'ABNB', 'REGN', 'KEY', 'IT', 'AWK', 'AIG', 'MHK', 'CDW', 'SYY', 'MKC', 'CINF', 'JBHT', 'PANW', 'JKHY', 'TXN', 'COP', 'SOLV', 'AVY', 'MAR', 'TRV', 'TECH', 'HST', 'SBAC', 'GM', 'HOLX', 'SPG', 'GE', 'COST', 'VICI', 'AXP', 'UPS', 'WFC', 'PARA', 'IPG', 'FANG', 'JNJ', 'CAT', 'VRSN', 'CHD', 'EA', 'TDY', 'WST', 'CSX', 'NVR', 'PH', 'AVGO', 'MTB', 'MLM', 'ABT', 'TXT', 'AOS', 'HCA', 'PNR', 'WBD', 'ORLY', 'MOH', 'KDP', 'AMGN', 'USB', 'TTWO', 'APO', 'TSCO', 'UNH', 'D', 'KR', 'WSM', 'MS', 'RCL', 'BK', 'ELV', 'MCO', 'NTRS', 'HUBB', 'AMT', 'KVUE', 'BXP', 'LH', 'DHI', 'ENPH', 'FTNT', 'GILD', 'IDXX', 'IR', 'YUM', 'SNPS', 'MSI', 'BG', 'EXE', 'PRU', 'FI', 'CMS', 'HSY', 'SPGI', 'KKR', 'BLDR', 'KMX', 

(0, 504)


Ticker,Date,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS


In [6]:
df_low=df.isna().sum().reset_index()
df_low=df_low[df_low[0]>0]
df_low.head()

,Ticker,0


In [7]:
df_low['Ticker'].unique()

array([], dtype=object)

In [8]:
df = df.drop(columns=df_low['Ticker'].unique())
print(df.shape)
df.head()

(0, 504)


Ticker,Date,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS


In [9]:
sp500 = yf.download('^GSPC', start=start_date, end=end_date)
sp500= sp500['Close'].reset_index()
df=df.merge(sp500, how='inner')
print(df.shape)
df.head()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['^GSPC']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


(0, 505)


Ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,Date,^GSPC


In [10]:
df=df[(df['Date']<'2019-12-31')]
df.head()

Ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,Date,^GSPC


In [11]:
df.drop(columns=['Date'],axis=1,inplace=True)
#df.set_index('Date', inplace=True)

df_returns = np.log(df / df.shift(1))

df_returns = df_returns.dropna()

#df_returns.reset_index(inplace=True)

print(df_returns.shape)
df_returns.head()

(0, 504)


Ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC


In [12]:
def min_cvar(df):
    header=df.columns.tolist()
    matrix_scenarios = [header, df.values]
    point_lower = [header, np.zeros(len(df.columns.tolist()))]
    matrix_budget =[header, np.ones(len(df.columns.tolist()))]


    allowExternal = True
    suppressMessages = False
    problem_name = "problem_min_var"
    problem_statement = "minimize\n\
    cvar_risk(0.99, matrix_scenarios)\n\
    Constraint: == 1\n\
    linear(matrix_budget)\n\
    Box: >= 0\n\
    "
    problem_dictionary = {'problem_name':problem_name, 'problem_statement':problem_statement,
                          'matrix_budget':matrix_budget, 
                          'matrix_cov':matrix_scenarios,
                          'point_lower':point_lower}
    
    res_noshorts = psg.psg_solver(problem_dictionary, allowExternal, suppressMessages)
    res=res_noshorts['point_problem_1']
    return res

In [13]:
w=min_cvar(df_returns)

SystemError: <built-in function psg_solver> returned a result with an exception set

In [ ]:
weights = pd.DataFrame({'Stocks': w[0],'Weights': w[1]})
weights['Stocks']=weights['Stocks'].str.upper()
weights.head()

In [ ]:
pos_weights=weights[weights['Weights']>0]
pos_weights.sort_values(by='Weights',inplace=True,ascending=False)
pos_weights.head()

In [ ]:
top=pos_weights.head(10)
bottom=pos_weights.tail(10)
bottom

In [ ]:
top

In [ ]:
def compute_risk_measures(returns_series,index_returns,THRESHOLD_Q,Q_LEVEL):
    cum_ret = returns_series.cumsum()
    drawdowns = cum_ret.expanding().max() - cum_ret

    edor = np.mean(np.maximum(drawdowns - THRESHOLD_Q, 0))

    cdar = np.mean(np.sort(drawdowns)[-int(Q_LEVEL * len(drawdowns)):])
    
    cvar_threshold = np.quantile(returns_series, 1-Q_LEVEL)
    cvar = -returns_series[returns_series <= cvar_threshold].mean()

    lpm1 = np.mean(np.maximum(0 - returns_series, 0))
    lpm2 = np.mean(np.maximum(0 - returns_series, 0) ** 2)
    lpm3 = np.mean(np.maximum(0 - returns_series, 0) ** 3)
    lpm4 = np.mean(np.maximum(0 - returns_series, 0) ** 4)

    max_drawdown = drawdowns.max()
    var = np.quantile(returns_series, 1-Q_LEVEL)
    volatility = returns_series.std() * np.sqrt(252)

    mask = index_returns < 0
    index_down = index_returns[mask]
    stock_down = returns_series[mask]
    combined = pd.concat([stock_down, index_down], axis=1).dropna()
    cov = np.cov(combined.iloc[:, 0], combined.iloc[:, 1])[0, 1]
    varr = np.var(combined.iloc[:, 1])
    downside_beta=cov / varr

    return [edor, cdar, cvar, lpm1, lpm2, lpm3, lpm4, max_drawdown, var ,volatility, downside_beta ]


In [ ]:
def risk_matrix(df,THRESHOLD_Q, Q_LEVEL):
    stocks=list(df.columns)
    stocks.remove('^GSPC')
    new_df = pd.DataFrame(
    {ticker: compute_risk_measures(df[ticker],df['^GSPC'],THRESHOLD_Q,Q_LEVEL) for ticker in stocks}).T

    new_df.columns = ["EDoR", "CDaR", "CVaR", "LPM1", "LPM2", "LPM3", "LPM4", "Max Drawdown","VaR","Volatility","Downside Beta"]


    return  new_df

In [ ]:
THRESHOLD_Q = 0.4  
Q_LEVEL = 0.05

risk_matrix_all=risk_matrix(df_returns,THRESHOLD_Q, Q_LEVEL)
rank=risk_matrix_all.rank()
rank=rank.reset_index()
rank

In [ ]:
top1=top.merge(rank,how='inner',left_on='Stocks',right_on='index')
bottom1=bottom.merge(rank,how='inner',left_on='Stocks',right_on='index')

top1['label']='Top'
bottom1['label']='Bottom'
df_final=pd.concat([top1,bottom1],axis=0)
df_final.drop(columns='index',inplace=True)
df_final

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df_sorted = df_final.sort_values(by='Weights', ascending=False)

top10 = df_sorted[df_sorted['label'] == 'Top']
bottom10 = df_sorted[df_sorted['label'] == 'Bottom']

sns.set(style="whitegrid")

fig, axes = plt.subplots(2, 2, figsize=(18, 10))

sns.barplot(data=top1, x='Stocks', y='Max Drawdown', ax=axes[0, 0],color='green')
axes[0, 0].set_title('Max Drawdown - Top')
axes[0, 0].tick_params(axis='x', rotation=90)

sns.barplot(data=bottom1, x='Stocks', y='Max Drawdown', ax=axes[0, 1],color='red')
axes[0, 1].set_title('Max Drawdown - Bottom')
axes[0, 1].tick_params(axis='x', rotation=90)

sns.barplot(data=top1, x='Stocks', y='Volatility', ax=axes[1, 0],color='green')
axes[1, 0].set_title('Volatility - Top')
axes[1, 0].tick_params(axis='x', rotation=90)

sns.barplot(data=bottom1, x='Stocks', y='Volatility', ax=axes[1, 1],color='red')
axes[1, 1].set_title('Volatility - Bottom')
axes[1, 1].tick_params(axis='x', rotation=90)

axes[0, 0].axhline(y=250, color='black', linestyle='--')
axes[0, 1].axhline(y=250, color='black', linestyle='--')
axes[1, 0].axhline(y=250, color='black', linestyle='--')
axes[1, 1].axhline(y=250, color='black', linestyle='--')


plt.tight_layout()
plt.show()


In [ ]:
rank

In [ ]:
df_ranked = rank.rename(columns={'index': 'Stocks'})

risk_weights = {'EDoR': 0.0,'CVaR': 0.2,'Max Drawdown': 0.2,'Volatility': 0.15,'LPM1': 0.05,'LPM2': 0.1,'LPM3': 0.1,
    'LPM4': 0.15,'Downside Beta': 0.05}

weighted_risks = df_ranked[list(risk_weights.keys())].mul(pd.Series(risk_weights), axis=1)
df_ranked['Composite_Risk_Score'] = weighted_risks.sum(axis=1)
df_ranked

In [ ]:
top

In [ ]:
newtop=top.merge(df_ranked,how='inner',left_on='Stocks',right_on='Stocks')
newbottom=bottom.merge(df_ranked,how='inner',left_on='Stocks',right_on='Stocks')

newtop['label']='Top'
newbottom['label']='Bottom'
df_final=pd.concat([newtop,newbottom],axis=0)
#df_final.drop(columns='index',inplace=True)
df_final

In [ ]:
newtop

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df_sorted = df_final.sort_values(by='Weights', ascending=False)

top10 = df_sorted[df_sorted['label'] == 'Top']
bottom10 = df_sorted[df_sorted['label'] == 'Bottom']

sns.set(style="whitegrid")

fig, axes = plt.subplots(1, 2, figsize=(18, 6))

sns.barplot(data=newtop, x='Stocks', y='Composite_Risk_Score', ax=axes[0], color='green')
axes[0].set_title('Composite Risk Score - Top')
axes[0].tick_params(axis='x', rotation=90)
axes[0].axhline(y=250, color='black', linestyle='--')

sns.barplot(data=newbottom, x='Stocks', y='Composite_Risk_Score', ax=axes[1], color='red')
axes[1].set_title('Composite Risk Score - Bottom')
axes[1].tick_params(axis='x', rotation=90)
axes[1].axhline(y=250, color='black', linestyle='--')

plt.tight_layout()
plt.show()
